## Initial Setup

1. Download the ZITMO version 2.0.8.0 reference sample from [MalwareBazaar](https://bazaar.abuse.ch/sample/f831d088c3a64b06843d970337f1c8877c9c1988d56720a7dee9d67efeaf78f0/).
2. Unpack the sample with [CAPE](https://github.com/kevoreilly/CAPEv2).

In [1]:
# Install required Python modules
import sys
!{sys.executable} -m pip install pefile

# Import required Python modules
import pprint
import re
import struct

import pefile

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Load the unpacked sample
fp = open("zitmo_2.0.8.0_unpacked.bin", "rb")
sample = fp.read()
fp.close()

pe = pefile.PE(data=sample)

## Decrypt Strings

Pseudocode:

    int __usercall CryptedStrings::_getA@<eax>(unsigned __int16 id@<ax>, int buffer@<edi>)
    {
      s *s; // eax
      int i; // ecx
      int i2; // esi
      char pb; // dl
      int result; // eax

      s = &dword_401588[2 * id];
      for ( i = 0; i < *&s->size; *(i2 + buffer) = pb )
      {
        i2 = i;
        pb = i ^ s->key ^ *(*&s->encodedString + i);
        ++i;
      }
      result = *&s->size;
      *(result + buffer) = 0;
      return result;
    }

Disassembly:

    .text:0040F333                               CryptedStrings___getA proc near
    .text:0040F333
    .text:0040F333 0F B7 C0                                      movzx   eax, ax
    .text:0040F336 8D 04 C5 88 15 40 00                          lea     eax, dword_401588[eax*8]
    .text:0040F33D 33 D2                                         xor     edx, edx
    .text:0040F33F 33 C9                                         xor     ecx, ecx
    .text:0040F341 66 3B 50 02                                   cmp     dx, [eax+2]
    .text:0040F345 73 19                                         jnb     short loc_40F360
    .text:0040F347 56                                            push    esi
    .text:0040F348
    .text:0040F348                               loc_40F348:
    .text:0040F348 8B 50 04                                      mov     edx, [eax+4]
    .text:0040F34B 0F B7 F1                                      movzx   esi, cx
    .text:0040F34E 8A 14 32                                      mov     dl, [edx+esi]
    .text:0040F351 32 10                                         xor     dl, [eax]
    .text:0040F353 32 D1                                         xor     dl, cl
    .text:0040F355 41                                            inc     ecx
    .text:0040F356 88 14 3E                                      mov     [esi+edi], dl
    .text:0040F359 66 3B 48 02                                   cmp     cx, [eax+2]
    .text:0040F35D 72 E9                                         jb      short loc_40F348
    .text:0040F35F 5E                                            pop     esi
    .text:0040F360
    .text:0040F360                               loc_40F360:
    .text:0040F360 0F B7 40 02                                   movzx   eax, word ptr [eax+2]
    .text:0040F364 C6 04 38 00                                   mov     byte ptr [eax+edi], 0
    .text:0040F368 C3                                            retn
    .text:0040F368                               CryptedStrings___getA endp

In [3]:
def decrypt_string(pe, id_):
    match = re.search(rb'\x8d\x04\xc5(?P<enc_strings_addr>[\s\S]{4})', 
                      pe.get_memory_mapped_image())
    enc_strings_addr = struct.unpack("I", match.group("enc_strings_addr"))[0]
    
    enc_string_addr = enc_strings_addr + 8 * id_
    enc_string_rva = enc_string_addr - pe.OPTIONAL_HEADER.ImageBase
    
    xor_key = pe.get_word_at_rva(enc_string_rva)
    encbuf_len = pe.get_word_at_rva(enc_string_rva + 2)
    encbuf_addr = pe.get_dword_at_rva(enc_string_rva + 4)
    encbuf_rva = encbuf_addr - pe.OPTIONAL_HEADER.ImageBase
    encbuf = pe.get_data(encbuf_rva, encbuf_len)
    
    plainbuf = b""
    for i in range(len(encbuf)):
        pb = (i ^ xor_key ^ encbuf[i]) & 0xff
        plainbuf += struct.pack("B", pb)
        
    return plainbuf.decode()


for i in range(178):
    plain_string = decrypt_string(pe, i)
    print(f"{i}: {plain_string}")

0: %s://%s:%s@%s/
1: ftp
2: pop3
3: anonymous
4: grabbed\%S_%02u_%02u_%02u.txt
5: Grabbed data from: %s

%S
6: %s%s
Referer: %S
User input: %s
Data:

%S
7: *EMPTY*
8: *UNKNOWN*
9: *FAILED TO PARSE "%s"*
10:  *BLOCKED*
11: Content-Type: %s

12: ZCID: %S

13: application/x-www-form-urlencoded
14: Mozilla\Firefox
15: user.js
16: profiles.ini
17: Profile%u
18: IsRelative
19: Path
20: user_pref("network.cookie.cookieBehavior", 0);
user_pref("privacy.clearOnShutdown.cookies", false);
user_pref("security.warn_viewing_mixed", false);
user_pref("security.warn_viewing_mixed.show_once", false);
user_pref("security.warn_submit_insecure", false);
user_pref("security.warn_submit_insecure.show_once", false);

21: user_pref("browser.startup.homepage", "%s");
user_pref("browser.startup.page", 1);

22: Software\Microsoft\Internet Explorer\Main
23: Start Page
24: Software\Microsoft\Internet Explorer\PhishingFilter
25: Enabled
26: EnabledV8
27: Software\Microsoft\Internet Explorer\Privacy
28: CleanCookies

## Extract Version

Pseudocode:

    ...
    version = 0x2000800;
    r = BinStorage::_addItem(binStorage, 4, 10003, 0x20000, &version);// #define SBCID_BOT_VERSION 10003
    ...
    
Disassembly:

    ...
    .text:0040CF4C 8D 45 F8                                      lea     eax, [ebp+version]
    .text:0040CF4F 50                                            push    eax
    .text:0040CF50 57                                            push    edi
    .text:0040CF51 68 13 27 00 00                                push    2713h
    .text:0040CF56 6A 04                                         push    4
    .text:0040CF58 5B                                            pop     ebx
    .text:0040CF59 8B C6                                         mov     eax, esi
    .text:0040CF5B C7 45 F8 00 08 00 02                          mov     [ebp+version], 2000800h
    .text:0040CF62 E8 57 8E 00 00                                call    BinStorage___addItem
    ...

In [4]:
match = re.search(rb'\x68\x13\x27\x00\x00[\s\S]{1,8}?\xc7\x45[\s\S](?P<version>[\s\S]{4})', 
                  pe.get_memory_mapped_image())
version_bytes = match.group("version")
version = ".".join([str(b) for b in version_bytes[::-1]])
print(version)

2.0.8.0


## Decrypt BaseConfig

Pseudocode:

    int __usercall Core::getBaseConfig@<eax>(int bc@<eax>)
    {
      __int64 v1; // rax
      int v2; // ecx
      int v3; // esi

      v1 = Mem::_copy(bc, dword_401B20, 784);
      v2 = dword_41A754 + dword_41A2A4 - v1;
      v3 = HIDWORD(v1);
      do
      {
        *v1 ^= *(v2 + v1);
        LODWORD(v1) = v1 + 1;
        --v3;
      }
      while ( v3 );
      return v1;
    }

Disassembly:

    .text:00403E4D                               Core__getBaseConfig proc near
    .text:00403E4D
    .text:00403E4D 56                                            push    esi
    .text:00403E4E BA 10 03 00 00                                mov     edx, 310h
    .text:00403E53 52                                            push    edx
    .text:00403E54 68 20 1B 40 00                                push    offset dword_401B20
    .text:00403E59 50                                            push    eax
    .text:00403E5A E8 F5 C5 00 00                                call    Mem___copy
    .text:00403E5F 8B 35 54 A7 41 00                             mov     esi, dword_41A754
    .text:00403E65 8B 0D A4 A2 41 00                             mov     ecx, dword_41A2A4
    .text:00403E6B 03 CE                                         add     ecx, esi
    .text:00403E6D 2B C8                                         sub     ecx, eax
    .text:00403E6F 8B F2                                         mov     esi, edx
    .text:00403E71
    .text:00403E71                               loc_403E71:
    .text:00403E71 8A 14 01                                      mov     dl, [ecx+eax]
    .text:00403E74 30 10                                         xor     [eax], dl
    .text:00403E76 40                                            inc     eax
    .text:00403E77 4E                                            dec     esi
    .text:00403E78 75 F7                                         jnz     short loc_403E71
    .text:00403E7A 5E                                            pop     esi
    .text:00403E7B C3                                            retn
    .text:00403E7B                               Core__getBaseConfig endp

In [5]:
match = re.search(rb'\xba(?P<encbuf_len>[\s\S]{2}\x00\x00)\x52\x68(?P<encbuf_addr>[\s\S]{4})',
                   pe.get_memory_mapped_image())

encbuf_len = struct.unpack("I", match.group("encbuf_len"))[0]
encbuf_addr = struct.unpack("I", match.group("encbuf_addr"))[0]
encbuf_rva = encbuf_addr - pe.OPTIONAL_HEADER.ImageBase
encbuf = pe.get_data(encbuf_rva, encbuf_len)

# #define PESECTION_OF_BASECONFIG_KEY 2
section = pe.sections[2]
xor_key = pe.get_data(section.VirtualAddress, encbuf_len)

base_config = b""
for i, eb in enumerate(encbuf):
    pb = eb ^ xor_key[i % len(xor_key)]
    base_config += struct.pack("B", pb)
    
print(base_config)

b'k$>\x0eu-\xeb\xf1Jo"\x9a\xacu\xd0\x8b#q\xf1&\x98\x81\xbc\x15\x1e2\x1a\xe9\n\x00<\x00h\xab/\xae=\xdc\xc8\x94\xb1JeT\xf8\'\xf5h\xaeq\x15Nj/>\xe8\x97\xf8=#\x7f\xb4\xe4\xd7\xa9\xa7\x00\xeee\xab\x8b\xc9\xa5\xbc"\xdf\x0e\xba\xb9\xa7Ok\x8d\xb9\xb2\xc0Z0\xf0Y\xceR\x06\x00\x08\x00\xd5 \x99\x16\x16\x1f\x9aV\xce\xceb\x00t\x00n\x001\x00\x00\x00\x96\xbaw\xb4\xd4]\xb6\xbf\x9b\x8d\xd4\x9d\x95\x1e\xe2nw\xc2^\x03\xd6\x87\xaa\x8b;\xa1\xbdoyj\x19z>\xa7\xfc=\xe9\r\xdb\xb2\x9b\x04\x0c\x1f\x91\xaa\xc3=\xb1\x1b?\x9ab\xf0E<\xa1\x05\xce\x87R\x80\xfda\xd3\x0eS\x93\xd7p&rr7O\xc0p\xcb$\xcd\xeb\xb2\xa6\xe1\x1d\x0f\xfb\xde\x00\x04\x00\x00\xe1\xae\xd5\xab}x\xe3~G\x8e\x7fIG\xae\x06G\xdb\x8a\xeddN\x10\xf9\xa2\xac\xee\xb2\xbb\xd5V\x15\x08\xd1\xfe\x89\xa1cl\x14\xc6\xe1p\xcf}B\xf2\x97}y\xed\x06K\x02\xbc\xa1\xa3\xba\x95S\x19\xf3S\x1b\x00\xc8x\x01\x8b\xaerN2L\x1d\xfcWhttp://doubletest4411.com/2x/b2/cfg_dtes2.bin\x00\xf4{\xff\x9d\xf4Im\x07\xdeZ\x11\xd5}\x997\xf9\x91\x1d\xad\xd7~\x08\x19IB\xaf\x8f\xe6\xa9\xfbc\x84\xc4\x0f<

## Parse BaseConfig

In [6]:
flags = struct.unpack("I", base_config[0xcc:0xcc+4])[0]
print(f"flags: 0x{flags:x}\n")

delayStats = struct.unpack("I", base_config[0x5c:0x5c+4])[0]
print(f"delayStats: {delayStats}\n")

defaultConfig = base_config[0x11c:0x11c+101].split(b"\x00")[0].decode().replace("http", "hxxp")
print(f"defaultConfig: {defaultConfig}\n")

baseKey = base_config[0x1bb:0x1bb+258]
print(f"baseKey: {baseKey}\n")

defaultBotnet = base_config[0x6a:0x6a+21].split(b"\x00\x00")[0].decode()
print(f"defaultBotnet: {defaultBotnet}\n")

delayReport = struct.unpack("I", base_config[0x2dc:0x2dc+4])[0]
print(f"delayReport: {delayReport}\n")

delayConfig = struct.unpack("I", base_config[0x1c:0x1c+4])[0]
print(f"delayConfig: {delayConfig}\n")

flags: 0x400

delayStats: 524294

defaultConfig: hxxp://doubletest4411.com/2x/b2/cfg_dtes2.bin

baseKey: b'nB\x14\xdd\xc4|\xaa\x1d\xb9Q\x86\n\x92\xd854\xeai\xee\x03\xa3\x13\'\xf9}\xd1V\x0e]2\x89J\xa2\xbdM\xe37 +\xe5\xba\xd3\xac\xd40\x96\xae\xa4\x06:%\xc9v\x05\x01\xc1c\x8b/\x81D\xfe\x08\x84^w\xf2\xa8K\x99y\xadtm\x80bf\t\xab\xb3\xaf\xbb\x02\x12\xe7\xe0\xa9Z\xf8\x8aRS\x00\xb2,\xcfk\xca\x16\x10\xa0\xc2\xd9\xce\xb0;\x82!\xd7\xe9\x90\x97\x9b\x87\xcdp=\xe8\x11\xa7\x7fs\x9c\\\xed\x85P#Y\xdf\xdeW\x8dj\xdb1\x19\x9aT\x1c\xb8\x95\x18\xb66d\x079\xbf\x93O\xa6I\xbe\xd6\xdc\xc0\x0bU\xcb\xf1\xb5\x1e\x04\x91\x8c\xc3\x8e-\xbc\x1a\xa1~3\xfc\x9e"\x0f\xec\xb4_N\xc7\xc8Fo>\xa5[\x0cq\xf7\x1f?x\xfbG\xfa\x17A\xc6`\xe2\x9f@\xef\xf3\xe4*\xfd8\xe1$\xd0)g\xd5\x94\xeb\xff\xf5EXl\x8fr\xf4\xb7&\xc5\xb1\r\xf6ua\x9dH<.\xcc\xd2L\xf0\xdaz\x88\x1behC{\x98\x83(\xe6\x15\x00\x00'

defaultBotnet: b t n 1

delayReport: 1310724

delayConfig: 3932170



## Download DynamicConfig

I haven't been able to find a DynamicConfig file for any sample of ZITMO. If you have any of these files in your archive, please reach out to curator@zeusmuseum.com / [@tildedennis](https://twitter.com/tildedennis).

## Decrypt DynamicConfig

Pseudocode:

    ...
    Mem::_copy(v7, rc4Key, 0x102u);
    Crypt::_rc4(v4, binStorage, dataSize);
    if ( dataSize )                             // inlined Crypt::_visualDecrypt
    {
      v5 = dataSize - 1;
      if ( dataSize != 1 )
      {
        do
        {
          *(v5 + binStorage) ^= *(v5 + binStorage - 1);
          --v5;
        }
        while ( v5 );
      }
    }

## Commands

Pseudocode:

    ...
    while ( 1 )
    {
    CryptedStrings::_getW(word_4014C8[4 * ci], commandNameBuffer);
    if ( !lstrcmpiW(*args, commandNameBuffer) )
      break;
    if ( ++ci >= 24 )
      goto LABEL_14;
    }
    ...
    
Disassembly:

    ...
    .text:0040FC1B                               loc_40FC1B:
    .text:0040FC1B 66 8B 04 FD C8 14 40 00                       mov     ax, ds:word_4014C8[edi*8]
    .text:0040FC23 8D B5 54 FD FF FF                             lea     esi, [ebp+commandNameBuffer]
    .text:0040FC29 E8 3B F7 FF FF                                call    CryptedStrings___getW
    .text:0040FC2E 8B C6                                         mov     eax, esi
    .text:0040FC30 50                                            push    eax
    .text:0040FC31 FF 33                                         push    dword ptr [ebx]
    .text:0040FC33 FF 15 D8 11 40 00                             call    ds:lstrcmpiW
    .text:0040FC39 85 C0                                         test    eax, eax
    .text:0040FC3B 74 08                                         jz      short loc_40FC45
    .text:0040FC3D 47                                            inc     edi
    .text:0040FC3E 83 FF 18                                      cmp     edi, 18h
    .text:0040FC41 72 D8                                         jb      short loc_40FC1B
    .text:0040FC43 EB 1F                                         jmp     short loc_40FC64
    ...

In [7]:
match = re.search(rb'\x66\x8b\x04\xfd(?P<indexes_addr>[\s\S]{4})', pe.get_memory_mapped_image())
indexes_addr = struct.unpack("I", match.group("indexes_addr"))[0]

for i in range(24):
    index_addr = indexes_addr + i * 8
    index_rva = index_addr - pe.OPTIONAL_HEADER.ImageBase
    enc_string_index = pe.get_word_at_rva(index_rva)
    
    plain_string = decrypt_string(pe, enc_string_index)
    print(f"{enc_string_index}: {plain_string}")

130: os_shutdown
131: os_reboot
132: bot_uninstall
133: bot_update
134: bot_bc_add
135: bot_bc_remove
136: bot_httpinject_disable
137: bot_httpinject_enable
138: fs_path_get
139: fs_search_add
140: fs_search_remove
141: user_destroy
142: user_logoff
143: user_execute
144: user_cookies_get
145: user_cookies_remove
146: user_certs_get
147: user_certs_remove
148: user_url_block
149: user_url_unblock
150: user_homepage_set
151: user_ftpclients_get
152: user_flashplayer_get
153: user_flashplayer_remove
